In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"seokyunkang","key":"4a58dfcf1617eb52ddfbab75ff11ccd0"}'}

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json
# 내가 참가한 대회 리스트 확인
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         91           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes         98           False  
store-sales-time-series-forecasting            2030-06-30 23:59:00  Getting Started  Knowledge        936           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        161           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       1788           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      14419           False  
house-pr

In [ ]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 21.6MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 8.17MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 31.5MB/s]


In [ ]:
# Import Module
import pandas as pd
import numpy as np
import os
from tensorflow.keras.layers import Dense
from tensorflow import keras

In [ ]:
# 모든 Data를 DataFrame의 형태로 dictionary에 넣어가지고 온다.
def import_Data(file_path):
  result = dict()
  for file in os.listdir(file_path):

    file_name = file[:-4]
    result[file_name] = pd.read_csv(file)
    
  return result

In [ ]:
# 해당 경로에 있는 모든 파일을 DataFrame으로 가지고 왔다.
file_path = "/content/Dataset/"
Rawdata_dict = import_Data(file_path)

In [ ]:
dict_key = list(Rawdata_dict.keys())
dict_key

['train', 'test', 'gender_submission']

In [ ]:
Rawdata_dict[dict_key[0]]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
# 흩어져 있는 데이터를 모아 하나의 Rawdata로 만든다.
def make_Rawdata(dict_data):

  dict_key = list(dict_data.keys())
  test_Dataset = pd.merge(dict_data["gender_submission"], dict_data["test"], how = 'outer', on = "PassengerId")
  Rawdata = pd.concat([dict_data["train"], test_Dataset])
  Rawdata.reset_index(drop=True, inplace=True)

  return Rawdata

In [ ]:
Rawdata = make_Rawdata(Rawdata_dict)
Rawdata

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
from copy import copy

def remove_columns(DF, remove_list):

  # 원본 정보 유지를 위해 copy하여, 원본 Data와의 종속성을 끊었다.
  result = copy(Rawdata)

  # PassengerId를 Index로 대치
  result.set_index("PassengerId", inplace = True)

  # 불필요한 column 제거
  for column in remove_list:

    del(result[column])

  return result

In [ ]:
remove_list = ["Name", "Ticket"]
DF_Hand = remove_columns(Rawdata, remove_list)
DF_Hand

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,NaN,S
2,1,1,female,38.0,1,0,71.2833,C85,C
3,1,3,female,26.0,0,0,7.9250,NaN,S
4,1,1,female,35.0,1,0,53.1000,C123,S
5,0,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...
1305,0,3,male,NaN,0,0,8.0500,NaN,S
1306,1,1,female,39.0,0,0,108.9000,C105,C
1307,0,3,male,38.5,0,0,7.2500,NaN,S


In [ ]:
DF_Hand.isnull().sum()

Survived       0
Pclass         0
Sex            0
Age          263
SibSp          0
Parch          0
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [ ]:
def missing_value(DF):
  # Cabin 변수를 제거
  del(DF["Cabin"])

  # 결측치가 있는 모든 행은 제거
  DF.dropna(inplace = True)

In [ ]:
#결측치 처리
missing_value(DF_Hand)
DF_Hand

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S
4,1,1,female,35.0,1,0,53.1000,S
5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
1301,1,3,female,3.0,1,1,13.7750,S
1303,1,1,female,37.0,1,0,90.0000,Q
1304,1,3,female,28.0,0,0,7.7750,S


In [ ]:
#문자 데이터 처리
DF_Hand["Sex"] = np.where(DF_Hand["Sex"].to_numpy() == "male", 0, 1)
DF_Hand["Embarked"] = np.where(DF_Hand["Embarked"].to_numpy() == "C", 0, np.where(DF_Hand["Embarked"].to_numpy() == "Q", 1, 2))
DF_Hand

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,0,22.0,1,0,7.2500,2
2,1,1,1,38.0,1,0,71.2833,0
3,1,3,1,26.0,0,0,7.9250,2
4,1,1,1,35.0,1,0,53.1000,2
5,0,3,0,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
1301,1,3,1,3.0,1,1,13.7750,2
1303,1,1,1,37.0,1,0,90.0000,1
1304,1,3,1,28.0,0,0,7.7750,2


In [ ]:
def one_hot_Encoding(data, column):

  # 한 변수 내 빈도
  freq = data[column].value_counts()

  # 빈도가 큰 순서로 용어 사전 생성
  vocabulary = freq.sort_values(ascending = False).index

  # DataFrame에 용어 사전 크기의 column 생성
  for word in vocabulary:
    
    new_column = column + "_" + str(word)
    data[new_column] = 0

  # 생성된 column에 해당하는 row에 1을 넣음
  for word in vocabulary:

    target_index = data[data[column] == word].index
    new_column = column + "_" + str(word)
    data.loc[target_index, new_column] = 1
  
  del(data[column])

In [ ]:
one_hot_Encoding(DF_Hand, 'Pclass')
one_hot_Encoding(DF_Hand, 'Sex')
one_hot_Encoding(DF_Hand, 'Embarked')
DF_Hand

,Survived,Age,SibSp,Parch,Fare,Pclass_3,Pclass_1,Pclass_2,Sex_0,Sex_1,Embarked_2,Embarked_0,Embarked_1
PassengerId,,,,,,,,,,,,,
1,0,22.0,1,0,7.2500,1,0,0,1,0,1,0,0
2,1,38.0,1,0,71.2833,0,1,0,0,1,0,1,0
3,1,26.0,0,0,7.9250,1,0,0,0,1,1,0,0
4,1,35.0,1,0,53.1000,0,1,0,0,1,1,0,0
5,0,35.0,0,0,8.0500,1,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301,1,3.0,1,1,13.7750,1,0,0,0,1,1,0,0
1303,1,37.0,1,0,90.0000,0,1,0,0,1,0,0,1
1304,1,28.0,0,0,7.7750,1,0,0,0,1,1,0,0


데이터셋 분리 및 표준화 
-------------------------

In [ ]:
# Label 생성
y_test, y_train = DF_Hand["Survived"][:300].to_numpy(), DF_Hand["Survived"][300:].to_numpy()

# Dataset 생성
del(DF_Hand["Survived"])
X_test, X_train = DF_Hand[:300].values, DF_Hand[300:].values

In [ ]:
def scale_adjust(X_test, X_train, C_number, key="min_max"):
  if key == "min_max":

    min_key = np.min(X_train[:, C_number])
    max_key = np.max(X_train[:, C_number])

    X_train[:, C_number] = (X_train[:, C_number] - min_key)/(max_key - min_key)
    X_test[:, C_number] = (X_test[:, C_number] - min_key)/(max_key - min_key)

  elif key == "norm":
    
    mean_key = np.mean(X_train[:, C_number])
    std_key = np.std(X_train[:, C_number])

    X_train[:, C_number] = (X_train[:, C_number] - mean_key)/std_key
    X_test[:, C_number] = (X_test[:, C_number] - mean_key)/std_key
  
  return X_test, X_train


In [ ]:
X_test, X_train = scale_adjust(X_test, X_train, 0, key="min_max")
X_test, X_train = scale_adjust(X_test, X_train, 3, key="min_max")

In [ ]:
X_test[0]

array([0.27345609, 1.        , 0.        , 0.01415106, 1.        ,
       0.        , 0.        , 1.        , 0.        , 1.        ,
       0.        , 0.        ])

In [ ]:
# 모델 생성
model = keras.Sequential()
model.add(Dense(128, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(16, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))

In [ ]:
# 모델 Compile
opt = keras.optimizers.Adam(learning_rate = 0.005)
model.compile(optimizer = opt,
              loss = "binary_crossentropy",
              metrics = ["binary_accuracy"])

In [ ]:
model.fit(X_train, y_train, epochs = 500)

Epoch 1/500
24/24 [==============================] - 1s 3ms/step - loss: 0.4748 - binary_accuracy: 0.8291
Epoch 2/500
24/24 [==============================] - 0s 3ms/step - loss: 0.3748 - binary_accuracy: 0.8654
Epoch 3/500
24/24 [==============================] - 0s 4ms/step - loss: 0.3347 - binary_accuracy: 0.8735
Epoch 4/500
24/24 [==============================] - 0s 3ms/step - loss: 0.3194 - binary_accuracy: 0.8843
Epoch 5/500
24/24 [==============================] - 0s 3ms/step - loss: 0.3264 - binary_accuracy: 0.8829
Epoch 6/500
24/24 [==============================] - 0s 3ms/step - loss: 0.3106 - binary_accuracy: 0.8843
Epoch 7/500
24/24 [==============================] - 0s 3ms/step - loss: 0.3088 - binary_accuracy: 0.8829
Epoch 8/500
24/24 [==============================] - 0s 3ms/step - loss: 0.3113 - binary_accuracy: 0.8789
Epoch 9/500
24/24 [==============================] - 0s 3ms/step - loss: 0.3146 - binary_accuracy: 0.8735
Epoch 10/500
24/24 [==========================

In [ ]:
pred = model.predict(X_test).reshape(X_test.shape[0])
pred = np.where(pred > 0.5, 1, 0)
accuracy = 1 - (np.where((pred - y_test) == 0, 0, 1).sum()/len(y_test))
print("Accuracy: ", accuracy)

Accuracy:  0.8133333333333334


In [ ]:
model.fit(X_train, y_train, epochs = 300)

Epoch 1/300
24/24 [==============================] - 0s 4ms/step - loss: 0.1875 - binary_accuracy: 0.9179
Epoch 2/300
24/24 [==============================] - 0s 4ms/step - loss: 0.1742 - binary_accuracy: 0.9112
Epoch 3/300
24/24 [==============================] - 0s 4ms/step - loss: 0.1725 - binary_accuracy: 0.9206
Epoch 4/300
24/24 [==============================] - 0s 4ms/step - loss: 0.1738 - binary_accuracy: 0.9112
Epoch 5/300
24/24 [==============================] - 0s 4ms/step - loss: 0.1618 - binary_accuracy: 0.9219
Epoch 6/300
24/24 [==============================] - 0s 5ms/step - loss: 0.1656 - binary_accuracy: 0.9206
Epoch 7/300
24/24 [==============================] - 0s 4ms/step - loss: 0.1738 - binary_accuracy: 0.9192
Epoch 8/300
24/24 [==============================] - 0s 3ms/step - loss: 0.1703 - binary_accuracy: 0.9098
Epoch 9/300
24/24 [==============================] - 0s 3ms/step - loss: 0.1784 - binary_accuracy: 0.9219
Epoch 10/300
24/24 [==========================

In [ ]:
pred = model.predict(X_test).reshape(X_test.shape[0])
pred = np.where(pred > 0.5, 1, 0)
accuracy = 1 - (np.where((pred - y_test) == 0, 0, 1).sum()/len(y_test))
print("Accuracy: ", accuracy)

Accuracy:  0.79


In [ ]:
from tensorflow.keras.layers import (Dense, Dropout, BatchNormalization)

In [ ]:
# 모델 생성
model = keras.Sequential()
model.add(BatchNormalization())
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.10))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.10))
model.add(Dense(32, activation = "relu"))
model.add(Dropout(0.10))
model.add(Dense(16, activation = "relu"))
model.add(Dropout(0.50))
model.add(Dense(1, activation = "sigmoid"))


opt = keras.optimizers.Adam(learning_rate = 0.005)
model.compile(optimizer = opt,
              loss = "binary_crossentropy",
              metrics = ["binary_accuracy"])

In [ ]:
model.fit(X_train, y_train, epochs = 500)

Epoch 1/500
24/24 [==============================] - 3s 6ms/step - loss: 0.5007 - binary_accuracy: 0.7752
Epoch 2/500
24/24 [==============================] - 0s 6ms/step - loss: 0.4063 - binary_accuracy: 0.8479
Epoch 3/500
24/24 [==============================] - 0s 9ms/step - loss: 0.4171 - binary_accuracy: 0.8587
Epoch 4/500
24/24 [==============================] - 0s 7ms/step - loss: 0.3622 - binary_accuracy: 0.8641
Epoch 5/500
24/24 [==============================] - 0s 9ms/step - loss: 0.3962 - binary_accuracy: 0.8708
Epoch 6/500
24/24 [==============================] - 0s 6ms/step - loss: 0.3835 - binary_accuracy: 0.8762
Epoch 7/500
24/24 [==============================] - 0s 5ms/step - loss: 0.3549 - binary_accuracy: 0.8748
Epoch 8/500
24/24 [==============================] - 0s 7ms/step - loss: 0.3926 - binary_accuracy: 0.8668
Epoch 9/500
24/24 [==============================] - 0s 7ms/step - loss: 0.3480 - binary_accuracy: 0.8694
Epoch 10/500
24/24 [==========================

In [ ]:
pred = model.predict(X_test).reshape(X_test.shape[0])
pred = np.where(pred > 0.5, 1, 0)
accuracy = 1 - (np.where((pred - y_test) == 0, 0, 1).sum()/len(y_test))
print("Accuracy: ", accuracy)

Accuracy:  0.8233333333333334
